In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Simulate Example LIDAR Data
# Generate 1000 samples of 360-degree LIDAR readings
np.random.seed(42)
num_samples = 1000
num_features = 360  # One for each degree
X = np.random.uniform(0, 10, size=(num_samples, num_features))  # Distances in meters

# Generate labels (0 = free path, 1 = obstacle)
# Rule: If any distance is < 0.5m, mark as obstacle (1), otherwise free path (0)
y = np.any(X < 0.5, axis=1).astype(int)

# Convert to DataFrame for inspection
data = pd.DataFrame(X)
data['label'] = y

# 2. Preprocess the Data
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 3. Define the Neural Network
class ObstacleDetector(nn.Module):
    def __init__(self):
        super(ObstacleDetector, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_features, 128),  # 360 inputs (LIDAR degrees)
            nn.ReLU(),
            nn.Linear(128, 2)  # Output: 2 classes (free path or obstacle)
        )

    def forward(self, x):
        return self.fc(x)

# Initialize the model, loss function, and optimizer
model = ObstacleDetector()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Train the Model
num_epochs = 20  # Number of training epochs
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()  # Clear gradients
    outputs = model(X_train_tensor)  # Forward pass
    loss = criterion(outputs, y_train_tensor)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

# 5. Evaluate the Model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = torch.argmax(test_outputs, dim=1)  # Get class predictions
    accuracy = (predictions == y_test_tensor).float().mean().item()

print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Optional: Save the Model
torch.save(model.state_dict(), "obstacle_detector.pth")


Epoch 1/20, Loss: 0.3145
Epoch 2/20, Loss: 0.0010
Epoch 3/20, Loss: 0.0000
Epoch 4/20, Loss: 0.0000
Epoch 5/20, Loss: 0.0000
Epoch 6/20, Loss: 0.0000
Epoch 7/20, Loss: 0.0000
Epoch 8/20, Loss: 0.0000
Epoch 9/20, Loss: 0.0000
Epoch 10/20, Loss: 0.0000
Epoch 11/20, Loss: 0.0000
Epoch 12/20, Loss: 0.0000
Epoch 13/20, Loss: 0.0000
Epoch 14/20, Loss: 0.0000
Epoch 15/20, Loss: 0.0000
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0000
Epoch 20/20, Loss: 0.0000
Test Accuracy: 100.00%
